In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install backtesting
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=d179aa8a78a5a9d9fe9d688a0b1bcb600e099052649078b9211eb3cb18eb414e
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [22]:
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG  # Google dataset is used for the daily price ticker data
import pandas as pd

# Custom Trend Magic Indicator given in the task sheet
def trend_magic(df, cci_period=20, atr_period=14, atr_multiplier=1.5):
    df['CCI'] = (df['Close'] - df['Close'].rolling(cci_period).mean()) / (0.015 * df['Close'].rolling(cci_period).std())
    df['ATR'] = df['Close'].rolling(atr_period).std() * atr_multiplier
    df['TrendMagic'] = df['Close'] + df['ATR']
    return df['TrendMagic']

# Define the Strategy with Entry and Exit Conditions
#using the stop loss and take profits level based on the ATR value at the current position
#dynamically adjusting the stop loss and take profits
class PerfectOrderTrendMagicDynamicATR(Strategy):
    ema_period = 45
    sma1_period = 90
    sma2_period = 180
    cci_period = 20
    atr_period = 14
    cci_bullish = 50
    cci_bearish = -50
    atr_multiplier_sl = 2
    atr_multiplier_tp = 3
    atr_multiplier = 1.5

    def init(self):
        # here I am defining the EMA45 ,sma90,sma180
        # using the self.I wrapper for the function
        self.ema45 = self.I(lambda x: pd.Series(x).ewm(span=self.ema_period).mean(), self.data.Close)
        self.sma90 = self.I(lambda x: pd.Series(x).rolling(window=self.sma1_period).mean(), self.data.Close)
        self.sma180 = self.I(lambda x: pd.Series(x).rolling(window=self.sma2_period).mean(), self.data.Close)

        # Calculating the commodity channel index and avg trade range
        self.cci = self.I(
            lambda x: (pd.Series(x) - pd.Series(x).rolling(self.cci_period).mean()) /
                      (0.015 * pd.Series(x).rolling(self.cci_period).std()),
            self.data.Close
        )
        self.atr = self.I(lambda x: pd.Series(x).rolling(self.atr_period).std() * self.atr_multiplier, self.data.Close)

        # finally we will be Determining the trend_magic custom indicator 
        
        ########################### This will be used to change color ####################################
        self.trend_magic = self.I(trend_magic, self.data.df)

    def next(self):
        # Define Perfect Order using moving avgs
        perfect_order_bullish = self.ema45 > self.sma90 > self.sma180
        perfect_order_bearish = self.ema45 < self.sma90 < self.sma180

        # CCI Classification for Trend
        bullish_trend = self.cci[-1] > self.cci_bullish
        bearish_trend = self.cci[-1] < self.cci_bearish

        # Entry Conditions
        if not self.position:
            # Entry for a long position
            if perfect_order_bullish and bullish_trend:
                stop_loss = max(0, self.data.Close[-1] - self.atr[-1] * self.atr_multiplier_sl)
                take_profit = max(0, self.data.Close[-1] + self.atr[-1] * self.atr_multiplier_tp)

                if take_profit > stop_loss:  # Ensure valid range
                    self.buy(sl=stop_loss, tp=take_profit)

            # Entry for a short position
            elif perfect_order_bearish and bearish_trend:
                stop_loss = max(0, self.data.Close[-1] + self.atr[-1] * self.atr_multiplier_sl)
                take_profit = max(0, self.data.Close[-1] - self.atr[-1] * self.atr_multiplier_tp)

                if stop_loss > take_profit:  # Ensure valid range
                    self.sell(sl=stop_loss, tp=take_profit)

        # Exit Conditions for Long positions
        elif self.position.is_long:
            exit_long = (
                # Break in Perfect Order
                not perfect_order_bullish or
                self.data.Close[-1] < self.trend_magic[-1] or
                self.cci[-1] < self.cci_bullish
            )
            if exit_long:
                self.position.close()

        # Exit Conditions for Short Trades
        elif self.position.is_short:
            exit_short = (
                # Break in Perfect Order
                not perfect_order_bearish or
                # Trend Magic reversal (price above Trend Magic)
                self.data.Close[-1] > self.trend_magic[-1] or
                # CCI rises above bearish threshold
                self.cci[-1] > self.cci_bearish
            )
            if exit_short:
                self.position.close()

# Initialize Backtest and Optimize
bt = Backtest(GOOG, PerfectOrderTrendMagicDynamicATR, cash=10_000, commission=0.002)

# Run the backtest with optimization on relevant parameters
stats = bt.optimize(
    atr_multiplier=[1.5,2.5,5],
    cci_period=range(10, 30, 10),
    ema_period=range(20, 60, 10),
    sma1_period=range(60, 120, 20),
    sma2_period=range(120, 200, 20),
    maximize='Equity Final [$]'
)

stats

Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   27.746741
Equity Final [$]                 18636.365169
Equity Peak [$]                  18636.365169
Return [%]                          86.363652
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    7.576734
Volatility (Ann.) [%]               13.852802
Sharpe Ratio                         0.546946
Sortino Ratio                        0.800092
Calmar Ratio                         0.219402
Max. Drawdown [%]                  -34.533636
Avg. Drawdown [%]                   -4.228299
Max. Drawdown Duration     1086 days 00:00:00
Avg. Drawdown Duration      127 days 00:00:00
# Trades                                  268
Win Rate [%]                        54.477612
Best Trade [%]                      17.315651
Worst Trade [%]                    -14.112104
Avg. Trade [%]                    

In [9]:
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG  # Google dataset is used for the daily price ticker data
import pandas as pd

# Custom Trend Magic Indicator
def trend_magic(df, cci_period=20, atr_period=14, atr_multiplier=1.5):
    df['CCI'] = (df['Close'] - df['Close'].rolling(cci_period).mean()) / (0.015 * df['Close'].rolling(cci_period).std())
    df['ATR'] = df['Close'].rolling(atr_period).std() * atr_multiplier
    df['TrendMagic'] = df['Close'] + df['ATR']
    return df['TrendMagic']

# Define the Strategy
class PerfectOrderTrendMagicDynamicATR(Strategy):
    ema_period = 45
    sma1_period = 90
    sma2_period = 180
    cci_period = 20
    atr_period = 14
    cci_bullish = 50
    cci_bearish = -50
    cci_reversal_threshold = -100  # Example threshold for reversal in bullish trends
    atr_multiplier_sl = 2
    atr_multiplier_tp = 3
    atr_multiplier = 1.5

    def init(self):
        # Moving averages
        self.ema45 = self.I(lambda x: pd.Series(x).ewm(span=self.ema_period).mean(), self.data.Close)
        self.sma90 = self.I(lambda x: pd.Series(x).rolling(window=self.sma1_period).mean(), self.data.Close)
        self.sma180 = self.I(lambda x: pd.Series(x).rolling(window=self.sma2_period).mean(), self.data.Close)

        # CCI and ATR
        self.cci = self.I(
            lambda x: (pd.Series(x) - pd.Series(x).rolling(self.cci_period).mean()) /
                      (0.015 * pd.Series(x).rolling(self.cci_period).std()),
            self.data.Close
        )
        self.atr = self.I(lambda x: pd.Series(x).rolling(self.atr_period).std() * self.atr_multiplier, self.data.Close)

        # Custom Trend Magic indicator
        self.trend_magic = self.I(trend_magic, self.data.df)

    def next(self):
        # Define Perfect Order using moving avgs
        perfect_order_bullish = self.ema45 > self.sma90 > self.sma180
        perfect_order_bearish = self.ema45 < self.sma90 < self.sma180

        # CCI Classification for Trend
        bullish_trend = self.cci[-1] > self.cci_bullish
        bearish_trend = self.cci[-1] < self.cci_bearish

        # Entry Conditions
        if not self.position:
            # Entry for a long position in a bullish trend with reversal threshold
            if perfect_order_bullish and (bullish_trend or self.cci[-1] < self.cci_reversal_threshold):
                stop_loss = max(0, self.data.Close[-1] - self.atr[-1] * self.atr_multiplier_sl)
                take_profit = max(0, self.data.Close[-1] + self.atr[-1] * self.atr_multiplier_tp)

                if take_profit > stop_loss:  # Ensure valid range
                    self.buy(sl=stop_loss, tp=take_profit)

            # Entry for a short position in a bearish trend with reversal threshold
            elif perfect_order_bearish and (bearish_trend or self.cci[-1] > abs(self.cci_reversal_threshold)):
                stop_loss = max(0, self.data.Close[-1] + self.atr[-1] * self.atr_multiplier_sl)
                take_profit = max(0, self.data.Close[-1] - self.atr[-1] * self.atr_multiplier_tp)

                if stop_loss > take_profit:  # Ensure valid range
                    self.sell(sl=stop_loss, tp=take_profit)

        # Exit Conditions for Long positions
        elif self.position.is_long:
            exit_long = (
                # Break in Perfect Order
                not perfect_order_bullish or
                self.data.Close[-1] < self.trend_magic[-1] or
                self.cci[-1] < self.cci_bullish
            )
            if exit_long:
                self.position.close()

        # Exit Conditions for Short Trades
        elif self.position.is_short:
            exit_short = (
                # Break in Perfect Order
                not perfect_order_bearish or
                self.data.Close[-1] > self.trend_magic[-1] or
                self.cci[-1] > self.cci_bearish
            )
            if exit_short:
                self.position.close()

# Initialize Backtest and Optimize
bt = Backtest(GOOG, PerfectOrderTrendMagicDynamicATR, cash=10_000, commission=0.002)

# Run the backtest with optimization on relevant parameters
stats = bt.optimize(
    atr_multiplier=[1.5, 2.5, 5],
    cci_period=range(7, 35, 7),
    ema_period=range(20, 60, 10),
    sma1_period=range(60, 120, 20),
    sma2_period=range(120, 200, 20),
    maximize='Return [%]'
)

stats
# bt.plot()


/opt/conda/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 576 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: failed to validate DatetimeTickFormatter(id='p1046', ...).days: expected a value of type str, got ['%d %b', '%a %d'] of type list

In [21]:
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG  # Google dataset is used for the daily price ticker data
import pandas as pd

# Custom Trend Magic Indicator
def trend_magic(df, cci_period=20, atr_period=14, atr_multiplier=1.5):
    df['CCI'] = (df['Close'] - df['Close'].rolling(cci_period).mean()) / (0.015 * df['Close'].rolling(cci_period).std())
    df['ATR'] = df['Close'].rolling(atr_period).std() * atr_multiplier
    df['TrendMagic'] = df['Close'] + df['ATR']
    return df['TrendMagic']

# Define the Strategy
class PerfectOrderTrendMagicDynamicATR(Strategy):
    ema_period = 45
    sma1_period = 90
    sma2_period = 180
    cci_period = 20
    atr_period = 14
    cci_bullish = 50
    cci_bearish = -50
    cci_reversal_threshold = -100  # Example threshold for reversal in bullish trends
    atr_multiplier_sl = 2
    atr_multiplier_tp = 3
    atr_multiplier = 1.5

    def init(self):
        # Moving averages
        self.ema45 = self.I(lambda x: pd.Series(x).ewm(span=self.ema_period).mean(), self.data.Close)
        self.sma90 = self.I(lambda x: pd.Series(x).rolling(window=self.sma1_period).mean(), self.data.Close)
        self.sma180 = self.I(lambda x: pd.Series(x).rolling(window=self.sma2_period).mean(), self.data.Close)

        # CCI and ATR
        self.cci = self.I(
            lambda x: (pd.Series(x) - pd.Series(x).rolling(self.cci_period).mean()) /
                      (0.015 * pd.Series(x).rolling(self.cci_period).std()),
            self.data.Close
        )
        self.atr = self.I(lambda x: pd.Series(x).rolling(self.atr_period).std() * self.atr_multiplier, self.data.Close)

        # Custom Trend Magic indicator
        self.trend_magic = self.I(trend_magic, self.data.df)

    def next(self):
        # Define Perfect Order using moving avgs
        perfect_order_bullish = self.ema45 > self.sma90 > self.sma180
        perfect_order_bearish = self.ema45 < self.sma90 < self.sma180

        # CCI Classification for Trend
        bullish_trend = self.cci[-1] > self.cci_bullish
        bearish_trend = self.cci[-1] < self.cci_bearish

        # Entry Conditions
        if not self.position:
            # Entry for a long position in a bullish trend with reversal threshold
            if perfect_order_bullish and (bullish_trend or self.cci[-1] < self.cci_reversal_threshold):
                stop_loss = max(0, self.data.Close[-1] - self.atr[-1] * self.atr_multiplier_sl)
                take_profit = max(0, self.data.Close[-1] + self.atr[-1] * self.atr_multiplier_tp)

                if take_profit > stop_loss:  # Ensure valid range
                    self.buy(sl=stop_loss, tp=take_profit)

            # Entry for a short position in a bearish trend with reversal threshold
            elif perfect_order_bearish and (bearish_trend or self.cci[-1] > abs(self.cci_reversal_threshold)):
                stop_loss = max(0, self.data.Close[-1] + self.atr[-1] * self.atr_multiplier_sl)
                take_profit = max(0, self.data.Close[-1] - self.atr[-1] * self.atr_multiplier_tp)

                if stop_loss > take_profit:  # Ensure valid range
                    self.sell(sl=stop_loss, tp=take_profit)

        # Exit Conditions for Long positions
        elif self.position.is_long:
            exit_long = (
                # Break in Perfect Order
                not perfect_order_bullish or
                self.data.Close[-1] < self.trend_magic[-1] or
                self.cci[-1] < self.cci_bullish
            )
            if exit_long:
                self.position.close()

        # Exit Conditions for Short Trades
        elif self.position.is_short:
            exit_short = (
                # Break in Perfect Order
                not perfect_order_bearish or
                self.data.Close[-1] > self.trend_magic[-1] or
                self.cci[-1] > self.cci_bearish
            )
            if exit_short:
                self.position.close()

from bokeh.plotting import show, figure
from bokeh.models import DatetimeTickFormatter
from bokeh.io import output_notebook

output_notebook()

# Function for fixed plot
def plot_fixed(results, df, indicators, colors, filename=None, plot_equity=True,
               plot_return=True, plot_pl=True, plot_volume=True, plot_drawdown=True,
               smooth_equity=False, relative_equity=False, superimpose=True,
               resample=False, reverse_indicators=False, show_legend=True, open_browser=True):
    from bokeh.models import DatetimeTickFormatter

    fig_ohlc = figure(width=900, height=400, x_axis_type='datetime')
    fig_ohlc.xaxis.formatter = DatetimeTickFormatter(
        days='%d %b',
        months='%m/%Y',
        hours='%H:%M',
        minutes='%H:%M'
    )

    # Add OHLC data to the figure
    fig_ohlc.line(df.index, df['Close'], line_width=2, legend_label='Close', color='black')
    
    # Add indicators to the figure with specified colors
    for ind, color in zip(indicators, colors):
        fig_ohlc.line(df.index, df[ind], legend_label=ind, color=color)
    
    show(fig_ohlc)

# Initialize Backtest and Optimize
bt = Backtest(GOOG, PerfectOrderTrendMagicDynamicATR, cash=10_000, commission=0.002)

# Run the backtest with optimization on relevant parameters
stats = bt.optimize(
    atr_multiplier=[1.5, 2.5, 5],
    cci_period=range(7, 35, 7),
    ema_period=range(20, 60, 10),
    sma1_period=range(60, 120, 20),
    sma2_period=range(120, 200, 20),
    maximize='Return [%]'
)

df = GOOG.copy() 

df['ema45'] = df['Close'].ewm(span=stats._strategy.ema_period).mean()
df['sma90'] = df['Close'].rolling(window=stats._strategy.sma1_period).mean()
df['sma180'] = df['Close'].rolling(window=stats._strategy.sma2_period).mean()
df['cci'] = (df['Close'] - df['Close'].rolling(stats._strategy.cci_period).mean()) / (0.015 * df['Close'].rolling(stats._strategy.cci_period).std())
df['atr'] = df['Close'].rolling(stats._strategy.atr_period).std() * stats._strategy.atr_multiplier
df['trend_magic'] = trend_magic(df, stats._strategy.cci_period, stats._strategy.atr_period, stats._strategy.atr_multiplier)

indicators = ['ema45', 'sma90', 'sma180', 'cci', 'atr', 'trend_magic']
colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown']

# Use the fixed plot function
stats
plot_fixed(stats, df, indicators, colors)


Loading BokehJS ...

/opt/conda/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 576 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   37.988827
Equity Final [$]                  17619.86389
Equity Peak [$]                  21862.425003
Return [%]                          76.198639
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    6.871187
Volatility (Ann.) [%]               16.894076
Sharpe Ratio                         0.406722
Sortino Ratio                        0.623972
Calmar Ratio                         0.205563
Max. Drawdown [%]                  -33.426236
Avg. Drawdown [%]                   -5.585787
Max. Drawdown Duration      883 days 00:00:00
Avg. Drawdown Duration      113 days 00:00:00
# Trades                                  363
Win Rate [%]                        51.515152
Best Trade [%]                      17.315651
Worst Trade [%]                    -14.112104
Avg. Trade [%]                    

In [36]:
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG  # Google dataset is used for the daily price ticker data
import pandas as pd

# Custom Trend Magic Indicator
def trend_magic(df, cci_period=20, atr_period=14, atr_multiplier=1.5):
    df['CCI'] = (df['Close'] - df['Close'].rolling(cci_period).mean()) / (0.015 * df['Close'].rolling(cci_period).std())
    df['ATR'] = df['Close'].rolling(atr_period).std() * atr_multiplier
    df['TrendMagic'] = df['Close'] + df['ATR']
    return df['TrendMagic']

# Define the Strategy
class PerfectOrderTrendMagicDynamicATR(Strategy):
    ema_period = 45
    sma1_period = 90
    sma2_period = 180
    cci_period = 20
    atr_period = 14
    cci_bullish = 50
    cci_bearish = -50
    cci_reversal_threshold = -100  # Example threshold for reversal in bullish trends
    atr_multiplier_sl = 2
    atr_multiplier_tp = 3
    atr_multiplier = 1.5

    def init(self):
        # Moving averages
        self.ema45 = self.I(lambda x: pd.Series(x).ewm(span=self.ema_period).mean(), self.data.Close)
        self.sma90 = self.I(lambda x: pd.Series(x).rolling(window=self.sma1_period).mean(), self.data.Close)
        self.sma180 = self.I(lambda x: pd.Series(x).rolling(window=self.sma2_period).mean(), self.data.Close)

        # CCI and ATR
        self.cci = self.I(
            lambda x: (pd.Series(x) - pd.Series(x).rolling(self.cci_period).mean()) /
                      (0.015 * pd.Series(x).rolling(self.cci_period).std()),
            self.data.Close
        )
        self.atr = self.I(lambda x: pd.Series(x).rolling(self.atr_period).std() * self.atr_multiplier, self.data.Close)

        # Custom Trend Magic indicator
        self.trend_magic = self.I(trend_magic, self.data.df)

    def next(self):
        # Define Perfect Order using moving avgs
        perfect_order_bullish = self.ema45 > self.sma90 > self.sma180
        perfect_order_bearish = self.ema45 < self.sma90 < self.sma180

        # CCI Classification for Trend
        bullish_trend = self.cci[-1] > self.cci_bullish
        bearish_trend = self.cci[-1] < self.cci_bearish

        # Entry Conditions
        if not self.position:
            # Entry for a long position in a bullish trend with reversal threshold
            if perfect_order_bullish and (bullish_trend or self.cci[-1] < self.cci_reversal_threshold):
                stop_loss = max(0, self.data.Close[-1] - self.atr[-1] * self.atr_multiplier_sl)
                take_profit = max(0, self.data.Close[-1] + self.atr[-1] * self.atr_multiplier_tp)

                if take_profit > stop_loss:  # Ensure valid range
                    self.buy(sl=stop_loss, tp=take_profit)

            # Entry for a short position in a bearish trend with reversal threshold
            elif perfect_order_bearish and (bearish_trend or self.cci[-1] > abs(self.cci_reversal_threshold)):
                stop_loss = max(0, self.data.Close[-1] + self.atr[-1] * self.atr_multiplier_sl)
                take_profit = max(0, self.data.Close[-1] - self.atr[-1] * self.atr_multiplier_tp)

                if stop_loss > take_profit:  # Ensure valid range
                    self.sell(sl=stop_loss, tp=take_profit)

        # Exit Conditions for Long positions
        elif self.position.is_long:
            exit_long = (
                # Break in Perfect Order
                not perfect_order_bullish or
                self.data.Close[-1] < self.trend_magic[-1] or
                self.cci[-1] < self.cci_bullish
            )
            if exit_long:
                self.position.close()

        # Exit Conditions for Short Trades
        elif self.position.is_short:
            exit_short = (
                # Break in Perfect Order
                not perfect_order_bearish or
                self.data.Close[-1] > self.trend_magic[-1] or
                self.cci[-1] > self.cci_bearish
            )
            if exit_short:
                self.position.close()

from bokeh.plotting import show, figure
from bokeh.models import DatetimeTickFormatter
from bokeh.io import output_notebook

output_notebook()

# Function for fixed plot
def plot_fixed(results, df, indicators, colors, filename=None, plot_equity=True,
               plot_return=True, plot_pl=True, plot_volume=True, plot_drawdown=True,
               smooth_equity=False, relative_equity=False, superimpose=True,
               resample=False, reverse_indicators=False, show_legend=True, open_browser=True):
    from bokeh.models import DatetimeTickFormatter

    fig_ohlc = figure(width=1200,height=600, x_axis_type='datetime')
    fig_ohlc.xaxis.formatter = DatetimeTickFormatter(
        days='%d %b',
        months='%m/%Y',
        hours='%H:%M',
        minutes='%H:%M'
    )

    # Add OHLC data to the figure
    fig_ohlc.line(df.index, df['Close'], line_width=2, legend_label='Close', color='black')
    
    # Add indicators to the figure with specified colors
    for ind, color in zip(indicators, colors):
        fig_ohlc.line(df.index, df[ind], legend_label=ind, color=color)
    
    # Add trade markers to the figure
#     for trade in trades:
#         if trade['is_long']:
#             fig_ohlc.circle(trade['entry'], trade['entry_price'], size=10, color='green', legend_label='Buy')
#             fig_ohlc.circle(trade['exit'], trade['exit_price'], size=10, color='red', legend_label='Sell')
#         else:
#             fig_ohlc.triangle(trade['entry'], trade['entry_price'], size=10, color='red', legend_label='Sell')
#             fig_ohlc.triangle(trade['exit'], trade['exit_price'], size=10, color='green', legend_label='Buy')
    
    show(fig_ohlc)

# Initialize Backtest and Optimize
bt = Backtest(GOOG, PerfectOrderTrendMagicDynamicATR, cash=10_000, commission=0.002)

# Run the backtest with optimization on relevant parameters
stats = bt.optimize(
    atr_multiplier=[1.5, 2.5, 5],
    cci_period=range(7, 35, 7),
    ema_period=range(20, 60, 10),
    sma1_period=range(60, 120, 20),
    sma2_period=range(120, 200, 20),
    maximize='Return [%]'
)

# Access the data and indicators from the backtest results
df = GOOG.copy()  # Use a copy of the original GOOG data

# Calculate the indicators manually and add them to the DataFrame using optimized parameters
ema_period = stats._strategy.ema_period
sma1_period = stats._strategy.sma1_period
sma2_period = stats._strategy.sma2_period
cci_period = stats._strategy.cci_period
atr_period = stats._strategy.atr_period
atr_multiplier = stats._strategy.atr_multiplier

df['ema45'] = df['Close'].ewm(span=ema_period).mean()
df['sma90'] = df['Close'].rolling(window=sma1_period).mean()
df['sma180'] = df['Close'].rolling(window=sma2_period).mean()
df['cci'] = (df['Close'] - df['Close'].rolling(cci_period).mean()) / (0.015 * df['Close'].rolling(cci_period).std())
df['atr'] = df['Close'].rolling(atr_period).std() * atr_multiplier
df['trend_magic'] = trend_magic(df, cci_period, atr_period, atr_multiplier)

# Extract trade details for plotting
# trades = []
# for t in stats._trades:
#     trades.append({
#         'entry': t.EntryTime,
#         'entry_price': t.EntryPrice,
#         'exit': t.ExitTime,
#         'exit_price': t.ExitPrice,
#         'is_long': t.Size > 0
#     })

# Define the indicators to be plotted and their colors
indicators = ['ema45', 'sma90', 'sma180', 'cci', 'atr', 'trend_magic']
colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown']

# Use the fixed plot function
plot_fixed(stats, df, indicators, colors)


Loading BokehJS ...

/opt/conda/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 576 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   37.988827
Equity Final [$]                  17619.86389
Equity Peak [$]                  21862.425003
Return [%]                          76.198639
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    6.871187
Volatility (Ann.) [%]               16.894076
Sharpe Ratio                         0.406722
Sortino Ratio                        0.623972
Calmar Ratio                         0.205563
Max. Drawdown [%]                  -33.426236
Avg. Drawdown [%]                   -5.585787
Max. Drawdown Duration      883 days 00:00:00
Avg. Drawdown Duration      113 days 00:00:00
# Trades                                  363
Win Rate [%]                        51.515152
Best Trade [%]                      17.315651
Worst Trade [%]                    -14.112104
Avg. Trade [%]                    